In [31]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

url = "https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=200&weight_min=0&weight_max=200&blood=&constellation=&keyword=&carrier="
response = requests.get(url)

In [32]:
soup = BeautifulSoup(response.text, 'lxml')
title = soup.title.string
print(title)

芸能人一覧 | キャスティング業務用データベース「タレメcasting NEXT」


In [33]:
link = soup.tbody.find_all("tr", class_="talent-data")
name = link[0].find("span", class_="fwb").string
height = link[0].find("td", class_="height").string
height = re.sub(r"\D", "", height)
weight = link[0].find("td", class_="weight").string
weight = re.sub(r"\D", "", weight)
link = link[0].find("img")["src"]
print(name, height, weight, link)

半野　雅 176 65 https://narrow.jp/img/upload/talent/M22-0485-211114.jpg


In [27]:
img = requests.get(link)
with open("img.jpg", "wb") as f:
    f.write(img.content)

In [60]:
ids = []
names = []
heights = []
weights = []
bmis = []
os.makedirs("talents_dataset", exist_ok=True)

In [ ]:
import time
first_time = True  # 既にtalents.csvにデータがある場合はFalseにする
last_save_idx = 0  # 最後に保存したID（途中で処理を中断した場合に使用）

for i in range(1, 34):
    url = f"https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=300&weight_min=0&weight_max=200&blood=&constellation=&keyword=&limit=100&carrier=&page={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    body = soup.tbody.find_all("tr", class_="talent-data")
    talents = body.copy()
    for j, talent in enumerate(talents):
        if (i - 1) * 100 + j > last_save_idx:
            ids.append((i - 1) * 100 + j)
            name = talent.find("span", class_="fwb").string.replace("\u3000", "")
            names.append(name)
            height = talent.find("td", class_="height").string
            height = int(re.sub(r"\D", "", height))
            heights.append(height)
            weight = talent.find("td", class_="weight").string
            weight = int(re.sub(r"\D", "", weight))
            weights.append(weight)
            bmi = weight / (height / 100) ** 2
            bmis.append(bmi)
            img_link = talent.find("img")["src"]
            img = requests.get(img_link)
            save_path = f"talents_dataset/{(i - 1) * 100 + j:04}.jpg"
            with open(save_path, "wb") as f:
                f.write(img.content)
            time.sleep(1)

    print(f"{i} pages done")
    df = pd.DataFrame(list(zip(ids, names, heights, weights, bmis)), columns=["id", "name", "height", "weight", "bmi"])
    if first_time:
        df.to_csv("talents_dataset/hoge.csv", index=False)
        first_time = False
    else:
        df.to_csv("talents_dataset/hoge.csv", mode="a", header=False, index=False)
    ids = []
    names = []
    heights = []
    weights = []
    bmis = []
    time.sleep(5)

In [ ]:
df = pd.DataFrame(list(zip(ids, names, heights, weights, bmis)), columns=["id", "name", "height", "weight", "bmi"])
df

In [54]:
df.to_csv("talents_dataset/talents.csv", mode="a", header=False, index=False)